### Import Modules

In [1]:
# Python Libraries
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Our Modules
from modules.multihead_attention import MultiHeadAttention
from modules.data_preprocessing import DataPreprocessing
from modules.input import Input

### Parameters

In [3]:
data_path = fr"D:\courses\Transformers\Data\eng_french.csv"
max_sentence_length = 20
batch_size = 16
embedding_dim = 512

### Execute Code

In [5]:
preprocess = DataPreprocessing(
    data_path = data_path,
    max_sentence_length = max_sentence_length,
    batch_size = batch_size
)
dataset, dataloader = preprocess.forward()

Loading Data from D:\courses\Transformers\Data\eng_french.csv
Creating English vocabulary...
Creating French vocabulary...
Converting English Sentences to integer...
Converting French Sentences to integer...
Preparing Dataset for Input...
Done.


In [6]:
inp = Input(
    max_sentence_length,
    vocab_size = preprocess.english_vocab_size,
    embedding_dim = embedding_dim
)

In [11]:
for batch in dataloader:
    english_batch, french_batch = batch
    
    encoder_input = inp.forward(english_batch)

    print(encoder_input.shape)
    
    break

torch.Size([16, 20, 512])


In [8]:
# class Encoder(nn.Module):
#     def __init__(self,
#                  x,
#                  batch_size, 
#                  seq_length,
#                  emb_size,
#                  ):
#         self.batch_size = batch_size
#         self.seq_length = seq_length
#         self.emb_size = emb_size
#         self.input_dim = (self.batch_size, self.seq_length, self.emb_size)
#         self.mask = torch.tril(torch.ones(self.seq_length,self.seq_length))
#         self.mask[self.mask == 0] = -torch.inf
#         self.mask[self.mask == 1] = 0
#         # self.mask = None   
#         self.x = x
        
#     def forward(self):
#         contextual_embeddings = self.multihead_attention(8)
#         return contextual_embeddings

#     def multihead_attention(self, num_heads):
#         num_heads = 8
#         mha = MultiHeadAttention(self.emb_size, num_heads)
#         contextual_embeddings = mha.multihead_attention(self.x, num_heads, self.mask)
#         return contextual_embeddings        

In [9]:
# batch_size = 16
# seq_length = 4056
# emb_size = 512
# num_encoders = 2
# input = torch.rand(batch_size, seq_length, emb_size)

In [10]:
# for i in range(num_encoders):
#     encoder = Encoder(input, batch_size, seq_length,emb_size)
#     input = encoder.forward()
# input.shape